In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
#pip install -o langchain-community faiss-cpu langchain-huggingface pymupdf tiktoken langchain-ollama python-dotenv

In [3]:
from langchain_community.document_loaders import PyMuPDFLoader
import glob
import os

# Folder containing your PDFs
pdf_folder = "C:/Users/DELL/Desktop/New folder (2)/souces"

# Get all PDF file paths
pdf_files = glob.glob(os.path.join(pdf_folder, "*.pdf"))

# Load all documents
documents = []
for pdf_path in pdf_files:
    loader = PyMuPDFLoader(pdf_path)
    docs = loader.load()
    documents.extend(docs)  # combine all loaded docs

print(f"Loaded {len(documents)} documents from {len(pdf_files)} PDFs")


Loaded 1440 documents from 3 PDFs


In [4]:
doc=documents[231]
print(doc.page_content)


7.8 / FORMAT-PRESERVING ENCRYPTION 231
As can be seen, XTS-AES mode, like CTR mode, is suitable for parallel oper-
ation. Because there is no chaining, multiple blocks can be encrypted or decrypted 
simultaneously. Unlike CTR mode, XTS-AES mode includes a nonce (the param-
eter i) as well as a counter (parameter j).
 7.8 FORMAT-PRESERVING ENCRYPTION
Format-preserving encryption (FPE) refers to any encryption technique that takes 
a plaintext in a given format and produces a ciphertext in the same format. For 
 example, credit cards consist of 16 decimal digits. An FPE that can accept this type of 
input would produce a ciphertext output of 16 decimal digits. Note that the ciphertext 
need not be, and in fact is unlikely to be, a valid credit card number. But it will have 
the same format and can be stored in the same way as credit card number plaintext.
A simple encryption algorithm is not format preserving, with the exception 
that it preserves the format of binary strings. For exampl

In [5]:
pdfs=[]
for root, dirs, files in os.walk("C:/Users/DELL/Desktop/New folder (2)/souces"):
    print (root, dirs, files)
    for file in files:
        if file.endswith(".pdf"):
            pdfs.append(os.path.join(root, file))

C:/Users/DELL/Desktop/New folder (2)/souces [] ['Cryptography and Network Security 7th E, William Stallings- Looserof7.pdf', 'Hacking.The.Art.of.Exploitation.2nd.Edition.pdf', 'Serious Cryptography  (by Jean‑Philippe Aumasson).pdf']


In [6]:
docs = []
for pdf in pdfs:
    loader = PyMuPDFLoader(pdf)
    pages = loader.load()
    docs.extend(loader.load())

In [7]:
len(docs)

1440

In [8]:
for i, doc in enumerate(docs):
    print(f"\n------Document--------")
    print(f"matadata: {doc.metadata}") 
    print(f"Content: {doc.page_content[:100]}...")  # Print first 100 characters of each document


------Document--------
matadata: {'producer': 'Acrobat Distiller 10.0.0 (Windows)', 'creator': 'Adobe InDesign CS6 (Macintosh)', 'creationdate': '2016-07-23T12:55:28+05:30', 'source': 'C:/Users/DELL/Desktop/New folder (2)/souces\\Cryptography and Network Security 7th E, William Stallings- Looserof7.pdf', 'file_path': 'C:/Users/DELL/Desktop/New folder (2)/souces\\Cryptography and Network Security 7th E, William Stallings- Looserof7.pdf', 'total_pages': 767, 'format': 'PDF 1.6', 'title': 'Binder.pdf', 'author': '006028', 'subject': '', 'keywords': '', 'moddate': '2017-03-19T11:05:09+00:00', 'trapped': '', 'modDate': 'D:20170319110509Z', 'creationDate': "D:20160723125528+05'30'", 'page': 0}
Content: ...

------Document--------
matadata: {'producer': 'Acrobat Distiller 10.0.0 (Windows)', 'creator': 'Adobe InDesign CS6 (Macintosh)', 'creationdate': '2016-07-23T12:55:28+05:30', 'source': 'C:/Users/DELL/Desktop/New folder (2)/souces\\Cryptography and Network Security 7th E, William Stallings

In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Optimized for speed: larger chunks = fewer embeddings to create
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,  # Increased from 1000 (2x larger chunks)
    chunk_overlap=50   # Reduced from 100 (less overlap)
)
chunks = text_splitter.split_documents(documents)
print(f"Created {len(chunks)} chunks (optimized for speed)")

Created 2236 chunks (optimized for speed)


In [10]:
for chunk in chunks[:1]:
    print(f"\n------Chunk--------")
    print(f"matadata: {chunk.metadata}") 
    print(f"Content: {chunk.page_content[:100]}...")  # Print first 100 characters of each chunk


------Chunk--------
matadata: {'producer': 'Acrobat Distiller 10.0.0 (Windows)', 'creator': 'Adobe InDesign CS6 (Macintosh)', 'creationdate': '2016-07-23T12:55:28+05:30', 'source': 'C:/Users/DELL/Desktop/New folder (2)/souces\\Cryptography and Network Security 7th E, William Stallings- Looserof7.pdf', 'file_path': 'C:/Users/DELL/Desktop/New folder (2)/souces\\Cryptography and Network Security 7th E, William Stallings- Looserof7.pdf', 'total_pages': 767, 'format': 'PDF 1.6', 'title': 'Binder.pdf', 'author': '006028', 'subject': '', 'keywords': '', 'moddate': '2017-03-19T11:05:09+00:00', 'trapped': '', 'modDate': 'D:20170319110509Z', 'creationDate': "D:20160723125528+05'30'", 'page': 1}
Content: CRYPTOGRAPHY AND 
NETWORK SECURITY
PRINCIPLES AND PRACTICE
SEVENTH EDITION
GLOBAL EDITION
William St...


In [11]:

len(chunks),len(documents),len(pdfs)

(2236, 1440, 3)

In [12]:
len(docs[2].page_content),len(chunks[2].page_content)

(2698, 744)

In [15]:
from langchain_ollama import OllamaEmbeddings

# Initialize Nomic Embed Text embeddings (faster & more accurate than BGE)
embeddings = OllamaEmbeddings(
    model="nomic-embed-text",
    base_url="http://localhost:11434"
)

# Test embedding on first chunk
test_embedding = embeddings.embed_query(chunks[0].page_content[:100])
print(f"Embedding dimension: {len(test_embedding)}")
print(f"First 10 values: {test_embedding[:10]}")
print("✓ Using Nomic Embed Text (faster & more accurate)")


Embedding dimension: 768
First 10 values: [-0.0011989336, 0.055569675, -0.21319975, -0.020772012, 0.040554833, 0.0064998292, -0.009853458, -0.023207089, -0.008076724, -0.026090603]
✓ Using Nomic Embed Text (faster & more accurate)


In [23]:
from langchain_community.vectorstores import FAISS
import os
from datetime import datetime
import time

vector_store_path = "faiss_index"

# Try loading existing vector store first
if os.path.exists(vector_store_path) and os.path.exists(f"{vector_store_path}/index.faiss"):
    print(f"Loading existing vector store from {vector_store_path}...")
    try:
        vector_store = FAISS.load_local(vector_store_path, embeddings, allow_dangerous_deserialization=True)
        print(f"✓ Vector store loaded from cache")
    except Exception as e:
        print(f"Cache corrupted: {e}")
        vector_store = None
else:
    vector_store = None

# If no cache or cache failed, create new vector store
if vector_store is None:
    print(f"\nCreating vector store for {len(chunks)} chunks...")
    print("Using smaller batches (25 chunks) with delays to prevent Ollama overload...\n")
    
    start = datetime.now()
    batch_size = 25  # Much smaller to avoid connection drops
    
    vector_store = None
    processed = 0
    
    for i in range(0, len(chunks), batch_size):
        batch_end = min(i + batch_size, len(chunks))
        batch = chunks[i:batch_end]
        
        print(f"Batch {i//batch_size + 1}/{(len(chunks)-1)//batch_size + 1}: chunks {i+1}-{batch_end}...", end=" ", flush=True)
        
        try:
            # Create or add to vector store
            if vector_store is None:
                # First batch - create the vector store
                vector_store = FAISS.from_documents(batch, embeddings)
            else:
                # Subsequent batches - add to existing store
                new_store = FAISS.from_documents(batch, embeddings)
                vector_store.add_documents(batch)
            
            processed = batch_end
            print(f"✓ ({processed}/{len(chunks)})")
            
            # Small delay between batches to let Ollama recover
            if batch_end < len(chunks):
                time.sleep(0.5)
                
        except Exception as e:
            print(f"✗ Error: {str(e)[:100]}")
            print(f"Processed {processed}/{len(chunks)} chunks before error")
            raise
    
    elapsed = (datetime.now() - start).total_seconds()
    print(f"\n✓ Vector store created in {elapsed//60:.0f}m {elapsed%60:.0f}s")
    
    # Save to disk
    print(f"Saving to {vector_store_path}...")
    vector_store.save_local(vector_store_path)
    print("✓ Saved")

# Validation
print("\n" + "="*60)
print("Vector Store Ready!")
print(f"  Documents: {len(vector_store.docstore._dict)}")

# Test search
test_results = vector_store.similarity_search("cryptography encryption", k=2)
print(f"\nQuick test - found {len(test_results)} relevant chunks")

Creating vector store for 2236 chunks...


ResponseError: do embedding request: Post "http://127.0.0.1:50660/embedding": read tcp 127.0.0.1:50672->127.0.0.1:50660: wsarecv: An existing connection was forcibly closed by the remote host. (status code: 500)

In [ ]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Initialize Ollama LLM (local, no API key needed)
llm = OllamaLLM(
    model="llama2-uncensored",
    base_url="http://localhost:11434"
)

# Create a retriever (FAISS vector store)
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

# Create prompt template
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""Answer the question based on the context provided.

Context: {context}

Question: {question}

Answer:"""
)

# Create RAG chain
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
)

print("✓ RAG Chain created with Ollama!")

# Test query
query = "What is cryptography?"
response = rag_chain.invoke(query)

print(f"Q: {query}")
print(f"A: {response}")
print("\n(This answer is based on your PDF sources!)")


NameError: name 'vector_store' is not defined